In [2]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv("Aug25_feeds.csv")

In [12]:
df_item = df[['id','source_name','country','preprocessed_titletopic_lda','description_lemtopic_lda']]
df_item['t_lda'] = df_item['preprocessed_titletopic_lda'].apply(lambda text: 't' + str(text)) 
df_item['d_lda'] = df_item['description_lemtopic_lda'].apply(lambda text: 'd' + str(text))
df_item.head()
df_item.drop(columns ='preprocessed_titletopic_lda', inplace = True)
df_item.drop(columns ='description_lemtopic_lda', inplace = True)
df_item.head()
#one-hot encoding for source_name
df_source_name = pd.get_dummies(df_item['source_name'])
df_country = pd.get_dummies(df_item['country'])
df_t_lda = pd.get_dummies(df_item['t_lda'])
df_d_lda = pd.get_dummies(df_item['d_lda'])

/Users/dhend/anaconda2/envs/Python3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/dhend/anaconda2/envs/Python3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
df_onehotencode = pd.concat([df_source_name, 
                               df_country, df_t_lda, df_d_lda], axis = 1)

In [14]:
df_onehotencode.head()

,BioPharma-Reporter,BioWorld,CA Institute for Regenerative Medicine,EU Commission Newsroom,FDA Press Releases,Fierce Biotech,GaBI Online,MD+DI,Med Device Online,MedDev & Diagnostic Industry,...,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [17]:
#normalization
df_onehotencode_normalized = df_onehotencode.apply(lambda x: x/np.sqrt(df_onehotencode.sum(axis = 1)))

In [18]:
df_onehotencode_normalized.head()

,BioPharma-Reporter,BioWorld,CA Institute for Regenerative Medicine,EU Commission Newsroom,FDA Press Releases,Fierce Biotech,GaBI Online,MD+DI,Med Device Online,MedDev & Diagnostic Industry,...,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df_item = pd.concat([df_item['id'], df_onehotencode_normalized], axis = 1)

In [25]:
df_item.to_csv('Aug25_onehotencoder_normalized.csv', index = False)

In [26]:
df_item = df_item.rename(columns={'id':'feed_master_id'})

In [54]:
#we get rating_csv from the feed_board table, in which we extracted 
#user_id, feed_master_id, and status
rating_csv = pd.read_csv('/Users/dhend/Rimsys/rimsys_dev_v2/public/Aug20_user_rating.csv')
#recode status: 2:muted->-1, 1:starred->1
rating_csv['rating'] = rating_csv['status'].apply(
    lambda x: -1 if x == 2 else 1)

In [55]:
rating_csv.head()

,user_id,feed_master_id,status,rating
0,342,17083,1,1
1,342,17087,2,-1
2,484,19415,1,1
3,484,19416,1,1
4,501,19422,1,1


In [56]:
rating = pd.pivot_table(rating_csv, values='rating',
                        index=['feed_master_id'],columns=['user_id'])

In [57]:
rating.head()

user_id,205,290,311,342,374,388,484,498,501,528,545,550,576,580,624,710,730
feed_master_id,,,,,,,,,,,,,,,,,
7961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
10081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
13537,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17083,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17087,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
rating_df = pd.DataFrame(columns = rating.columns,index = df_item['feed_master_id'])

In [59]:
rating_df.head()

user_id,205,290,311,342,374,388,484,498,501,528,545,550,576,580,624,710,730
feed_master_id,,,,,,,,,,,,,,,,,
16552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
rating_df.update(rating)

In [61]:
rating_df.head()

user_id,205,290,311,342,374,388,484,498,501,528,545,550,576,580,624,710,730
feed_master_id,,,,,,,,,,,,,,,,,
16552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
rating_df = rating_df.replace(np.nan,0)

In [64]:
rating_df.to_csv("template.csv")

In [65]:
#caculating user profile based on idf_item and rating_df
idf_np = df_item.iloc[:,1:].to_numpy()
rating_np = rating_df.to_numpy()
user_profile = []
for i in range(rating_np.shape[1]):
    temp = []
    for j in range(idf_np.shape[1]):
        score = np.dot(rating_np[:,i],idf_np[:,j]).sum()
        temp.append(score)
    user_profile.append(temp)
user_profile = pd.DataFrame(user_profile)

In [66]:
user_profile.head()

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,-0.5,0.0,1.0,2.5,0.0,...,2.0,0.5,0.5,1.5,1.0,-0.5,2.5,-0.5,0.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.5,...,0.0,-0.5,0.0,0.0,0.5,0.0,0.5,0.0,0.0,0.5
3,-0.5,0.0,0.0,0.0,0.5,0.5,0.0,0.0,-1.0,0.0,...,-0.5,0.5,0.0,0.0,-1.0,-0.5,2.0,0.5,0.0,0.5
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
#caculate idf:
document_frequency = df_item.iloc[:,1:].astype(bool).sum(axis = 0)
idf = (len(df_item)/document_frequency).apply(np.log10)
idf_df_item = df_item.iloc[:,1:].mul(idf.values)


In [80]:
#creating prediction table
idf_df_item_np = idf_df_item.to_numpy()
user_profile_np = user_profile.to_numpy()
prediction=[]
for i in range(len(user_profile_np)):
    temp = []
    for j in range(len(idf_df_item_np)):
        score = (np.dot(user_profile_np[i],idf_df_item_np[j])).sum()
        temp.append(score)
    prediction.append(temp)
prediction=pd.DataFrame(prediction)
prediction=prediction.T
predict=prediction.to_numpy()
final = pd.DataFrame(data=predict,index=df_item['feed_master_id'],columns=rating_df.columns)

In [87]:
final.max()

user_id
205    1.592619
290    5.002428
311    1.323373
342    2.597918
374    0.216773
388    2.175428
484    7.070509
498    1.163423
501    4.590939
528    1.914197
545    3.638258
550    3.811713
576    3.329379
580    1.314472
624    4.150255
710    1.570557
730    0.806028
dtype: float64

In [88]:
final.min()

user_id
205   -0.234667
290   -0.911218
311   -0.706599
342   -1.698508
374   -0.256918
388    0.000000
484   -1.635564
498   -0.673611
501    0.000000
528   -0.446858
545    0.000000
550   -3.939315
576   -0.213714
580   -0.277533
624   -1.027307
710   -2.318204
730    0.000000
dtype: float64

In [90]:
final.to_csv('/Users/dhend/Rimsys/rimsys_dev_v2/public/Aug20_prediction.csv')